In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.cm as cm

In [ ]:
# Mueller-Brown potential in 2d

class MuellerPotential:
    def __init__(self, *argv):

        self.dim = 2
        self.a = [-1, -1, -6.5, 0.7]
        self.b = [0, 0, 11, 0.6]
        self.c = [-10, -10, -6.5, 0.7]
        self.A = [-200, -100, -170, 15]
        self.xc = [1, 0, -0.5, -1]
        self.yc = [0, 0.5, 1.5, 1]

        self.x_domain = [-1.8, 1.2]
        self.y_domain = [-0.5, 2.2]
        self.v_min_max = [-130, 20]
        self.contour_levels = [-130, -100, -80, -60, -40, -20, 0.0]
        self.density_max = 0.35
        
    def V(self, x):
        s = 0
        for i in range(4):
            dx = x[0] - self.xc[i]
            dy = x[1] - self.yc[i]
            s += self.A[i] * np.exp(self.a[i] * dx**2 + self.b[i] * dx * dy + self.c[i] * dy**2)
        return s
        
    def gradV(self, x):
        s = 0
        dVx = 0
        dVy = 0
        for i in range(4):
            dx = x[0] - self.xc[i]
            dy = x[1] - self.yc[i]            
            dVx += self.A[i] * (2 * self.a[i] * dx + self.b[i] * dy) * np.exp(self.a[i] * dx**2 + self.b[i] * dx * dy + self.c[i] * dy**2)
            dVy += self.A[i] * (self.b[i] * dx + 2 * self.c[i] * dy) * np.exp(self.a[i] * dx**2 + self.b[i] * dx * dy + self.c[i] * dy**2)
        return np.array((dVx, dVy))

# sample the SDE using Euler-Maruyama scheme
def sample(pot, beta=1.0, delta_t = 0.001, N=10000, seed=42):
    rng = np.random.default_rng(seed=seed)
     
    X = [-0.6, 1.2]
    dim = 2 
    traj = []
    save = 100
    tlist = []
    for i in tqdm(range(N)):
        b = rng.normal(size=(dim,))
        X = X - pot.gradV(X) * delta_t + np.sqrt(2 * delta_t/beta) * b
        if i % save==0:
            traj.append(X)
            tlist.append(i * delta_t)

    return np.array(tlist), np.array(traj)

In [ ]:
pot = MuellerPotential()  

In [ ]:
#assert pot.dim == 2, "dimension must be 2"

nx = 100
ny = 150

dx = (pot.x_domain[1] - pot.x_domain[0]) / nx
dy = (pot.y_domain[1] - pot.y_domain[0]) / ny

gridx = np.linspace(pot.x_domain[0], pot.x_domain[1], nx)
gridy = np.linspace(pot.y_domain[0], pot.y_domain[1], ny)
x_plot = np.outer(gridx, np.ones(ny)) 
y_plot = np.outer(gridy, np.ones(nx)).T 

x2d = np.concatenate((x_plot.reshape(nx * ny, 1), y_plot.reshape(nx * ny, 1)), axis=1)

pot_on_grid = np.array([pot.V(x) for x in x2d]).reshape(nx, ny)

print ( "min and max values of potential: (%.4f, %.4f)" % (pot_on_grid.min(), pot_on_grid.max()) )

fig = plt.figure(figsize=(9,5))
ax0 = fig.add_subplot(1, 1, 1)

im = ax0.pcolormesh(x_plot, y_plot, pot_on_grid, cmap='coolwarm', vmin=pot.v_min_max[0], vmax=pot.v_min_max[1])
contours = ax0.contour(x_plot, y_plot, pot_on_grid,  pot.contour_levels)
ax0.clabel(contours, inline=True, fontsize=13,colors='black')

ax0.set_aspect('equal')
ax0.set_xlabel(r'$x_1$',fontsize=20)
ax0.set_ylabel(r'$x_2$',fontsize=20, rotation=0)
ax0.tick_params(axis='both', labelsize=20)

ax0.set_xticks([-1.5, -1.0, -0.5, 0, 0.5, 1.0])
ax0.set_yticks([-0.5, 0, 0.5, 1.0, 1.5, 2.0])
ax0.set_xlim([pot.x_domain[0], pot.x_domain[1]])
ax0.set_ylim([pot.y_domain[0], pot.y_domain[1]])

ax0.set_title("Meuller-Brown potential",fontsize=20)
cbar = fig.colorbar(im, ax=ax0, shrink=1.0)
cbar.ax.tick_params(labelsize=15)

plt.show()

In [ ]:
tlist, trajectory = sample(pot, beta=0.1, delta_t=0.0002, N=5000000)

print (trajectory.shape)

In [ ]:
fig = plt.figure(figsize=(12,10))

ax0 = fig.add_subplot(2, 2, 1)
ax1 = fig.add_subplot(2, 2, 2)
ax2 = fig.add_subplot(2, 2, 3)
ax3 = fig.add_subplot(2, 2, 4)

nx = ny = 200

dx = (pot.x_domain[1] - pot.x_domain[0]) / nx
dy = (pot.y_domain[1] - pot.y_domain[0]) / ny
gridx = np.linspace(pot.x_domain[0], pot.x_domain[1], nx)
gridy = np.linspace(pot.y_domain[0], pot.y_domain[1], ny)
x_plot = np.outer(gridx, np.ones(ny)) 
y_plot = np.outer(gridy, np.ones(nx)).T 

x2d = np.concatenate((x_plot.reshape(nx * ny, 1), y_plot.reshape(nx * ny, 1)), axis=1)

h = np.histogram2d(trajectory[:,0], trajectory[:,1], bins=[nx, ny], range=[[pot.x_domain[0],pot.x_domain[1]],[pot.y_domain[0],pot.y_domain[1]]])[0]
s = sum(sum(h))
im = ax0.imshow(h.T / (s * dx * dy), origin = "lower",                 #h.T, origin = "lower", \
                extent=[pot.x_domain[0],pot.x_domain[1],pot.y_domain[0], pot.y_domain[1]], \
                cmap=cm.jet, vmin=0, vmax=pot.density_max)
fig.colorbar(im, ax=ax0)
ax0.set_xlim([pot.x_domain[0], pot.x_domain[1]])
ax0.set_ylim([pot.y_domain[0], pot.y_domain[1]])
ax0.set_title('empirical density')

pot_on_grid = np.array([pot.V(x) for x in x2d]).reshape(nx, ny)

contours = ax1.contour(x_plot, y_plot, pot_on_grid, levels=pot.contour_levels, cmap='coolwarm')

ax1.scatter(trajectory[:,0], trajectory[:,1], alpha=0.5, c='k', s=4)
ax1.set_xlim([pot.x_domain[0], pot.x_domain[1]])
ax1.set_ylim([pot.y_domain[0], pot.y_domain[1]])
ax1.set_title('trajectory')

ax2.plot(tlist, trajectory[:,0])
ax2.set_ylim([pot.x_domain[0], pot.x_domain[1]])
ax2.set_title('x coodinate along trajectory')
ax3.plot(tlist, trajectory[:,1])
ax3.set_ylim([pot.y_domain[0], pot.y_domain[1]])
ax3.set_title('y coodinate along trajectory')

plt.show()